# Good Reads Choice Award Books Web Scrape

prototyping using BeautifulSoup and requests for webscraping.

In [12]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import re
import time
import pandas as pd
from tqdm import tqdm
import csv
import pandas as pd

Testing out the initial web page.

In [2]:
link = 'https://www.goodreads.com/book/show/39863520-how-to-date-men-when-you-hate-men'
page = urllib.request.urlopen(link)

soup = bs(page, 'html.parser')

#soup.prettify()

In [3]:
soup.title

<title>How to Date Men When You Hate Men by Blythe Roberson</title>

Get a list of URLs for 15 comments about a book.

In [4]:
# url = 'https://www.goodreads.com/book/reviews/39863520-how-to-date-men-when-you-hate-men?rating=&text_only=true'
# r = requests.get(url).content
# prefix = 'https://www.goodreads.com/review/show/'
# [prefix + x.split('\\')[0] for x in str(r).split(prefix)][1:]

## Get URLs for the top books 2019

### We started by getting the url for every genre category

In [5]:
url = 'https://www.goodreads.com/choiceawards/best-fiction-books-2019'

In [6]:
r = urllib.request.urlopen(url)
soup = bs(r, 'html.parser')

In [8]:
#this extracts the url for every genre category of 2019 choice awards
soup.find_all('a', {'class':'categoriesList__categoryLink categoriesList__categoryLink--current'})
every_genre_list = []
for a in soup.find_all('a',{'class':"categoriesList__categoryLink categoriesList__categoryLink--current"},  href=True):
    every_genre_list.append('https://www.goodreads.com' + (a['href']))


In [9]:
every_genre_list

['https://www.goodreads.com/choiceawards/best-mystery-thriller-books-2019',
 'https://www.goodreads.com/choiceawards/best-historical-fiction-books-2019',
 'https://www.goodreads.com/choiceawards/best-fantasy-books-2019',
 'https://www.goodreads.com/choiceawards/best-romance-books-2019',
 'https://www.goodreads.com/choiceawards/best-science-fiction-books-2019',
 'https://www.goodreads.com/choiceawards/best-horror-books-2019',
 'https://www.goodreads.com/choiceawards/best-humor-books-2019',
 'https://www.goodreads.com/choiceawards/best-nonfiction-books-2019',
 'https://www.goodreads.com/choiceawards/best-memoir-autobiography-books-2019',
 'https://www.goodreads.com/choiceawards/best-history-biography-books-2019',
 'https://www.goodreads.com/choiceawards/best-science-technology-books-2019',
 'https://www.goodreads.com/choiceawards/best-food-cookbooks-2019',
 'https://www.goodreads.com/choiceawards/best-graphic-novels-comics-2019',
 'https://www.goodreads.com/choiceawards/best-poetry-books

### then we get the urls for every book in all categories

In [17]:
# # Run this code to retrieve all good r
# book_list_2019 = []
# for i in tqdm(every_genre_list):
#     url = i
#     r = urllib.request.urlopen(url)
#     soup = bs(r, 'html.parser')
#     for a in soup.find_all('a',{"class":"pollAnswer__bookLink"},  href=True):
#         book_list_2019.append('https://www.goodreads.com' + (a['href']))
# df = pd.DataFrame(book_list_2019)
# book_list_2019 = df.to_csv('book_list_2019')

100%|██████████| 19/19 [00:58<00:00,  3.09s/it]


In [55]:
book_list_2019 = pd.read_csv('book_list_2019').drop('Unnamed: 0', axis=1)
# this is all urls for the best books of 2019
len(book_list_2019) # 380
book_list_2019 = book_list_2019.squeeze().tolist() # turn CSV back into a list

## Extract Relevant Data from Choice Award books

### Create functions to retrieve dates

In [80]:
def get_date(soup):
    link = soup
    page = urllib.request.urlopen(link)
    soup = bs(page, "html.parser")
    get_date = soup.find('div', {'id':"details"})
    get_date = get_date.contents[3].get_text()
    get_date = get_date.replace('\n', '')
    get_date = re.findall(r'((?<=Published        ).+\b)(\d{4})', get_date)[0]
#     get_date = re.find
    get_date = get_date[0].rstrip() + ', ' + get_date[1]
    get_date = get_date.rsplit('by', 2)[0].rstrip() # gets rid of extra text after published date

#     all(r'((?<=Published        ).+\b)(\d{4})', get_date)[0]
    return get_date
def get_month_day(soup):
    get_month_day = get_date(soup)[0]
    return get_month_day
def get_month(soup):
    get_month = get_date(soup)[0].split(' ')[0]
    return get_month
def get_year(soup):
    get_year = get_date(soup)[1]
    return get_year

get_date(book_list_2019[1])

'November 20th 2018'

### Get all book details using this function below

In [84]:
def get_description(book_links):
    book_info = []
    for url in tqdm(book_links):
        r = requests.get(url)
        soup = bs(r.text)
        get_descrip = soup.find('div', {'id':"description"})
        div = soup.find('div', id='description')
        description = div.text
        description = description[1:-9]
        
        get_title = soup.find('h1',{"id":"bookTitle"})
        get_title = get_title.get_text().replace('\n', '').replace('  ', '')
        
        get_genre = soup.find('a', {'class':"actionLinkLite bookPageGenreLink"})
        get_genre = get_genre.get_text()
        
        get_rating = soup.find('span',{"itemprop":"ratingValue"})
        get_rating = get_rating.get_text().replace('\n', '').replace(' ', '')
        
        get_author = soup.find('span', {'itemprop':"name"})
        get_author = get_author.get_text()
        
        get_publisher = soup.find('div', {'id':"details"})
        get_publisher = get_publisher.contents[3].get_text()
        get_publisher = get_publisher.replace('\n', '')
        get_publisher = re.findall(r'(?<=by ).+\b', get_publisher)[0]
        get_publisher = get_publisher.rsplit('(',2)[0].rstrip()
        
        get_pages = soup.find('span', {'itemprop':"numberOfPages"})
        get_pages = get_pages.get_text().split(' ')[0]
        
        get_format = soup.find('span', {'itemprop':"bookFormat"})
        get_format = get_format.get_text()
        
        book_info.append({'title': get_title, 'author': get_author, 'description':description, 'genre': get_genre, 'rating': get_rating,
           'publisher': get_publisher, 'pages':get_pages, 'format': get_format, 'date': get_date(url)
                         ,'url': url})
    book_info = pd.DataFrame(book_info)
    book_info.to_csv('all_book_info.csv')
        
    return book_info 
    

# get_description(book_list_2019) # run this code to save a csv file of all book info